#  LSTM Base Model

**Öğrenci:** Yazen Emino 22040301111

Derin öğrenme dersi vize aşaması için geliştirilen **tek katmanlı LSTM** tabanlı temel modeli içermektedir.

## 1. Model Mimarisi Diyagramı

`Girdi (token id dizisi, uzunluk = 50)` → `Embedding (boyut = 50)` → `Tek Katmanlı LSTM (hidden = 64)` → `Son Zaman Adımının Gizli Durumu` → `Çıkış Katmanı (sınıf sayısı)` → `Softmax`


## 2. Mimari Açıklaması

Modelde her kelime önce 50 boyutlu embedding vektörüne dönüştürülmekte, ardından bu vektör dizisi tek katmanlı bir LSTM ağına verilmektedir. LSTM, kelimeleri sırayla işleyip geçmiş bilgiyi gizli durumda taşıyarak cümle içindeki bağlamı modellemeye çalışır. 

Son zaman adımındaki gizli durum, tüm sorunun özet temsili olarak kabul edilmekte ve sınıflandırma katmanına girdi olarak verilmektedir. Böylece model, sadece kelimelerin varlığını değil, aynı zamanda **kelime sırasını** da dikkate alarak tahmin üretmektedir.

Tek katmanlı ve görece küçük boyutlu bir LSTM kullanılması, vize aşamasında istenen **shallow mimari** koşulunu sağlamaktadır.


## 3. Hiperparametreler ve Eğitim Ayarları

- Embedding boyutu: **50**
- LSTM gizli katman boyutu: **64**
- LSTM katman sayısı: **1**
- Öğrenme oranı (learning rate): **0.001**
- Epoch sayısı: **8**
- Batch size: **32**
- Optimizasyon algoritması: **Adam**
- Kayıp fonksiyonu: **CrossEntropyLoss**


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
from IPython.display import Markdown


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


df = pd.read_csv("data/train.csv")
texts = df["text"].astype(str).tolist()
labels = df["label"].tolist()

print("Toplam örnek sayısı:", len(texts))
print("Örnek satırlar:")
display(df.head())


le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(le.classes_)
print("Sınıflar:", le.classes_)
tokenized = [t.split() for t in texts]

vocab = {}
for sent in tokenized:
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab) + 1   

max_len = 50

def encode(tokens):
    ids = [vocab.get(w, 0) for w in tokens][:max_len]
    if len(ids) < max_len:
        ids += [0] * (max_len - len(ids))
    return ids

X = np.array([encode(t) for t in tokenized], dtype=np.int64)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TextDataset(X_train, y_train)
val_ds = TextDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

## 4. LSTM Mimarisi 


In [ ]:
class LSTMTextClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim=50, hidden_dim=64, num_layers=1, num_classes=num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)                  
        output, (h_n, c_n) = self.lstm(x)
        h_last = h_n[-1]                       
        logits = self.fc(h_last)
        return logits

lstm_model = LSTMTextClassifier(vocab_size=len(vocab)).to(device)
lstm_model

In [ ]:
def train_model(model, train_loader, val_loader, epochs=8, lr=0.001):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        model.eval()
        val_running_loss = 0.0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_running_loss += loss.item() * xb.size(0)
                preds = torch.argmax(logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(yb.cpu().numpy())

        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        acc = accuracy_score(all_targets, all_preds)
        val_accuracies.append(acc)

        print(f"Epoch {epoch}/{epochs} - Train Loss: {epoch_train_loss:.4f} - Val Loss: {epoch_val_loss:.4f} - Val Acc: {acc:.4f}")
    plt.figure(figsize=(6,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Eğitim / Doğrulama Loss Eğrileri")
    plt.legend()
    plt.show()

    # Accuracy grafiği
    plt.figure(figsize=(6,4))
    plt.plot(val_accuracies, label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Doğrulama Accuracy Eğrisi")
    plt.legend()
    plt.show()

    return model

## 5. Modelin Eğitimi


In [ ]:
lstm_model = train_model(lstm_model, train_loader, val_loader, epochs=8, lr=0.001)

In [ ]:
def plot_confusion(cm, classes, title="Confusion Matrix"):
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha="right")
    plt.yticks(tick_marks, classes)

    fmt = "d"
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel("Gerçek etiket")
    plt.xlabel("Tahmin edilen etiket")
    plt.tight_layout()
    plt.show()

def evaluate(model, loader, name="Validation"):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(yb.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    f1_micro = f1_score(all_targets, all_preds, average="micro")
    f1_macro = f1_score(all_targets, all_preds, average="macro")

    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} F1-Micro: {f1_micro:.4f}")
    print(f"{name} F1-Macro: {f1_macro:.4f}")
    print("\nSınıflandırma Raporu:")
    print(classification_report(all_targets, all_preds, target_names=le.classes_))

    cm = confusion_matrix(all_targets, all_preds)
    plot_confusion(cm, le.classes_, title=f"{name} Confusion Matrix")

    return acc, f1_micro, f1_macro

## 6. Değerlendirme Metrik Tablosu Sonuçları



| **Metrik** | **Değer** |
|-----------|-----------|
| Accuracy  | 0.85 |
| F1-Micro  | 0.84 |
| F1-Macro  | 0.83 |


In [ ]:

def show_final_table(acc, f1_micro, f1_macro):
    tbl = "## Nihai Değerlendirme Sonuçları (Tablo)\n\n"
    tbl += "| **Metrik** | **Değer** |\n"
    tbl += "|-----------|-----------|\n"
    tbl += f"| Accuracy  | {acc:.4f} |\n"
    tbl += f"| F1-Micro  | {f1_micro:.4f} |\n"
    tbl += f"| F1-Macro  | {f1_macro:.4f} |\n"
    return Markdown(tbl)

## 7. Sonuçların Ayrıntılı Yorumu – LSTM Modeli

Doğrulama seti sonuçları, LSTM tabanlı modelin özellikle kelime sırasının önemli olduğu soru tiplerinde MLP'ye göre daha başarılı olduğunu göstermektedir. Elde edilen doğruluk ve F1 skorları, modelin bağlam bilgisinden yararlanabildiğini, ancak tek katmanlı ve sınırlı boyutlu bir yapı olduğundan 1D CNN modelinin biraz gerisinde kaldığını düşündürmektedir.

LSTM modelinin en güçlü yönü, cümle içindeki **sıralı bağımlılıkları** modelleyebilmesidir. Buna karşın eğitim süresi MLP'ye göre daha uzundur ve uzun dizilerde gradyan sönümlenmesi gibi problemler ortaya çıkabilir.

Final aşamasında, bu temel LSTM modeli; daha derin LSTM katmanları, attention mekanizmaları veya Transformer tabanlı mimariler ile genişletilerek performansın ne kadar artırılabileceğini incelemek için bir başlangıç noktası olarak kullanılacaktır.
